In [1]:
import pandas as pd
import numpy as np
import datetime
import re
from collections import Counter

In [2]:
reviewsdf = pd.read_csv('data/reviewsdf.csv')

In [3]:
reviewsdf.drop(columns=['Unnamed: 0'], inplace=True)

In [4]:
reviewsdf.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5528 entries, 0 to 5527
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          5528 non-null   object 
 1   num_reviews    5528 non-null   int64  
 2   release_date   5526 non-null   object 
 3   orig_price     5528 non-null   float64
 4   been_sale      5528 non-null   int64  
 5   review_rating  5528 non-null   object 
 6   rating_val     5521 non-null   float64
 7   num_languages  2102 non-null   float64
 8   tags           5528 non-null   object 
 9   num_tags       5528 non-null   float64
 10  genres         5528 non-null   object 
 11  num_genres     5528 non-null   float64
 12  publisher      5523 non-null   object 
 13  top_pubs       5528 non-null   object 
dtypes: float64(5), int64(2), object(7)
memory usage: 604.8+ KB


# initializing final dataframe
## removing rows with nulls

In [5]:
reviewsdf = reviewsdf[reviewsdf.rating_val.isna()==False]

In [6]:
reviewsdf = reviewsdf[reviewsdf.release_date.isna()==False]
reviewsdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5519 entries, 0 to 5527
Data columns (total 14 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   title          5519 non-null   object 
 1   num_reviews    5519 non-null   int64  
 2   release_date   5519 non-null   object 
 3   orig_price     5519 non-null   float64
 4   been_sale      5519 non-null   int64  
 5   review_rating  5519 non-null   object 
 6   rating_val     5519 non-null   float64
 7   num_languages  2101 non-null   float64
 8   tags           5519 non-null   object 
 9   num_tags       5519 non-null   float64
 10  genres         5519 non-null   object 
 11  num_genres     5519 non-null   float64
 12  publisher      5514 non-null   object 
 13  top_pubs       5519 non-null   object 
dtypes: float64(5), int64(2), object(7)
memory usage: 646.8+ KB


## creating year-released column

In [7]:
reviewsdf.release_date = pd.to_datetime(reviewsdf.release_date)

In [8]:
reviewsdf['year_released'] = reviewsdf.release_date.dt.year

In [9]:
reviewsdf.head(3)

,title,num_reviews,release_date,orig_price,been_sale,review_rating,rating_val,num_languages,tags,num_tags,genres,num_genres,publisher,top_pubs,year_released
0,Sea of Thieves,111078,2020-06-03,39.99,1,Very Positive,8.0,13.0,"['Adventure', 'Multiplayer', 'Open World', 'Pi...",20.0,"['Action', 'Adventure']",2.0,Xbox Game Studios,other,2020
1,It Takes Two,11813,2021-03-26,39.99,0,Overwhelmingly Positive,9.0,12.0,"['Co-op', 'Adventure', '3D Platformer', 'Puzzl...",20.0,"['Action', 'Adventure', 'Hazelight']",3.0,Electronic Arts,Electronic Arts,2021
2,Valheim,216651,2021-02-02,19.99,0,Overwhelmingly Positive,9.0,12.0,"['Open World Survival Craft', 'Online Co-Op', ...",20.0,"['Action', 'Adventure', 'Indie', 'RPG', 'Early...",6.0,Coffee Stain Publishing,other,2021


## languages filled column
filled with median

In [10]:
reviewsdf['languages_filled'] = reviewsdf.num_languages.fillna(reviewsdf.num_languages.median())

In [11]:
reviewsdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5519 entries, 0 to 5527
Data columns (total 16 columns):
 #   Column            Non-Null Count  Dtype         
---  ------            --------------  -----         
 0   title             5519 non-null   object        
 1   num_reviews       5519 non-null   int64         
 2   release_date      5519 non-null   datetime64[ns]
 3   orig_price        5519 non-null   float64       
 4   been_sale         5519 non-null   int64         
 5   review_rating     5519 non-null   object        
 6   rating_val        5519 non-null   float64       
 7   num_languages     2101 non-null   float64       
 8   tags              5519 non-null   object        
 9   num_tags          5519 non-null   float64       
 10  genres            5519 non-null   object        
 11  num_genres        5519 non-null   float64       
 12  publisher         5514 non-null   object        
 13  top_pubs          5519 non-null   object        
 14  year_released     5519 n

# quick setting final df for later

In [13]:
# stting final df here, so i can just merge columns as needed
# finaldf = reviewsdf[['num_reviews', 'year_released', 'orig_price', 'been_sale',
#                     'rating_val', 'num_languages', 'num_tags', 'num_genres',]]

In [12]:
finaldf.head(3)

NameError: name 'finaldf' is not defined

# publisher into dummies
resetting top_pubs to be 15 top publishers instead of 10

In [11]:
reviewsdf.drop(columns=['top_pubs'],inplace=True)

In [12]:
reviewsdf.head(3)

,title,num_reviews,release_date,orig_price,been_sale,review_rating,rating_val,num_languages,tags,num_tags,genres,num_genres,publisher,year_released,languages_filled
0,Sea of Thieves,111078,2020-06-03,39.99,1,Very Positive,8.0,13.0,"['Adventure', 'Multiplayer', 'Open World', 'Pi...",20.0,"['Action', 'Adventure']",2.0,Xbox Game Studios,2020,13.0
1,It Takes Two,11813,2021-03-26,39.99,0,Overwhelmingly Positive,9.0,12.0,"['Co-op', 'Adventure', '3D Platformer', 'Puzzl...",20.0,"['Action', 'Adventure', 'Hazelight']",3.0,Electronic Arts,2021,12.0
2,Valheim,216651,2021-02-02,19.99,0,Overwhelmingly Positive,9.0,12.0,"['Open World Survival Craft', 'Online Co-Op', ...",20.0,"['Action', 'Adventure', 'Indie', 'RPG', 'Early...",6.0,Coffee Stain Publishing,2021,12.0


In [13]:
top_pub_list = list(reviewsdf.publisher.value_counts().head(14).keys()) #top 15 publishers


def big_publisher(string):
    if string in (top_pub_list):
        return string
    else:
        return 'other'
    
    

In [14]:
#creating new column with just top 15 publishers and other, to create less dummy columns
reviewsdf['top_pubs'] = reviewsdf.publisher.apply(big_publisher)

In [15]:
reviewsdf.head()

,title,num_reviews,release_date,orig_price,been_sale,review_rating,rating_val,num_languages,tags,num_tags,genres,num_genres,publisher,year_released,languages_filled,top_pubs
0,Sea of Thieves,111078,2020-06-03,39.99,1,Very Positive,8.0,13.0,"['Adventure', 'Multiplayer', 'Open World', 'Pi...",20.0,"['Action', 'Adventure']",2.0,Xbox Game Studios,2020,13.0,Xbox Game Studios
1,It Takes Two,11813,2021-03-26,39.99,0,Overwhelmingly Positive,9.0,12.0,"['Co-op', 'Adventure', '3D Platformer', 'Puzzl...",20.0,"['Action', 'Adventure', 'Hazelight']",3.0,Electronic Arts,2021,12.0,Electronic Arts
2,Valheim,216651,2021-02-02,19.99,0,Overwhelmingly Positive,9.0,12.0,"['Open World Survival Craft', 'Online Co-Op', ...",20.0,"['Action', 'Adventure', 'Indie', 'RPG', 'Early...",6.0,Coffee Stain Publishing,2021,12.0,other
3,Horizon Zero Dawn™ Complete Edition,45704,2020-08-07,49.99,1,Mostly Positive,7.0,20.0,"['Adventure', 'Open World', 'Action', 'Female ...",20.0,"['Action', 'Adventure', 'RPG']",3.0,"PlayStation Mobile, Inc.",2020,20.0,other
4,Phasmophobia,256136,2020-09-18,13.99,0,Overwhelmingly Positive,9.0,25.0,"['Horror', 'Online Co-Op', 'Multiplayer', 'Psy...",20.0,[],0.0,Kinetic Games,2020,25.0,other


In [16]:
pub_dummies = pd.get_dummies(reviewsdf.top_pubs, drop_first=True)

reviewsdf = pd.merge(reviewsdf, pub_dummies, left_index=True, right_index=True)

In [17]:
reviewsdf.head(3)

,title,num_reviews,release_date,orig_price,been_sale,review_rating,rating_val,num_languages,tags,num_tags,...,Electronic Arts,Kagura Games,Nightdive Studios,Paradox Interactive,SEGA,Square Enix,THQ Nordic,Ubisoft,Xbox Game Studios,other
0,Sea of Thieves,111078,2020-06-03,39.99,1,Very Positive,8.0,13.0,"['Adventure', 'Multiplayer', 'Open World', 'Pi...",20.0,...,0,0,0,0,0,0,0,0,1,0
1,It Takes Two,11813,2021-03-26,39.99,0,Overwhelmingly Positive,9.0,12.0,"['Co-op', 'Adventure', '3D Platformer', 'Puzzl...",20.0,...,1,0,0,0,0,0,0,0,0,0
2,Valheim,216651,2021-02-02,19.99,0,Overwhelmingly Positive,9.0,12.0,"['Open World Survival Craft', 'Online Co-Op', ...",20.0,...,0,0,0,0,0,0,0,0,0,1


In [18]:
reviewsdf.shape

(5519, 30)

# tags
next we'll use the code from df_editing to find the top 30 most used tags, create columns for them
and then change the columns if the column has that tag

In [19]:
regex = re.compile('[^1-9a-zA-Z]')


In [20]:
tags_list=[]
# for items in (reviewsdf.iloc[0].tags.split()):
#     list.append(regex.sub('',items))

In [21]:
tags_list

[]

In [22]:
#makes a list of all tags
def megalist(tags):
    for items in (tags.split()):
        tags_list.append(regex.sub('',items)) 

In [23]:
reviewsdf.tags.apply(megalist)

0       None
1       None
2       None
3       None
4       None
        ... 
5523    None
5524    None
5525    None
5526    None
5527    None
Name: tags, Length: 5519, dtype: object

In [24]:
tags_list[:10]

['Adventure',
 'Multiplayer',
 'Open',
 'World',
 'Pirates',
 'Action',
 'Coop',
 'Online',
 'CoOp',
 'Sailing']

In [25]:
# takes the top 30 most common tags
tags_count = Counter(tags_list).most_common()[:30]

In [26]:
top_tags = []
for tags in tags_count:
    top_tags.append(tags[0])

In [27]:
top_tags

['Singleplayer',
 'Indie',
 'Action',
 'Adventure',
 'Multiplayer',
 'Strategy',
 'RPG',
 'Casual',
 'Simulation',
 'Atmospheric',
 'Horror',
 '2D',
 'Soundtrack',
 'Great',
 'Story',
 'Rich',
 'World',
 'Shooter',
 'Puzzle',
 'TurnBased',
 'Open',
 'FirstPerson',
 'Survival',
 'Funny',
 'Coop',
 'Fantasy',
 'Early',
 'Access',
 'Platformer',
 'Local']

## manually created dummies for top 30 tags

In [28]:
#created a column for each of the 30 tags, with tags suffix
for tags in top_tags:
    reviewsdf[tags+'_tag'] = 0

In [29]:
reviewsdf.shape

(5519, 60)

## code to fill in each manually created dummy column

In [73]:

for items in top_tags:
    reviewsdf.loc[reviewsdf.tags.str.contains(items),[items+'_tag']] = 1

In [77]:
reviewsdf.Action_tag.sum()

2694

# genres
the same code abobe will be applied to genres.

In [31]:
genres_list =[]

def megalist_genre(genres):
    for items in (genres.split()):
        genres_list.append(regex.sub('',items)) # function was called once with tags_list, once with genres_list

reviewsdf.genres.apply(megalist_genre)

0       None
1       None
2       None
3       None
4       None
        ... 
5523    None
5524    None
5525    None
5526    None
5527    None
Name: genres, Length: 5519, dtype: object

In [38]:
#top 30 genres 
genres_count = Counter(tags_list).most_common()[:30]

In [43]:
top_genres = []
for genre in genres_count:
    top_genres.append(genre[0])

In [48]:
top_genres

['Singleplayer',
 'Indie',
 'Action',
 'Adventure',
 'Multiplayer',
 'Strategy',
 'RPG',
 'Casual',
 'Simulation',
 'Atmospheric',
 'Horror',
 '2D',
 'Soundtrack',
 'Great',
 'Story',
 'Rich',
 'World',
 'Shooter',
 'Puzzle',
 'TurnBased',
 'Open',
 'FirstPerson',
 'Survival',
 'Funny',
 'Coop',
 'Fantasy',
 'Early',
 'Access',
 'Platformer',
 'Local']

In [49]:
#creating top 30 genre columns with genre suffix
for genre in top_genres:
    reviewsdf[genre+'_genre'] = 0

In [64]:
for items in top_genres:
    reviewsdf.loc[reviewsdf.genres.str.contains(items),[items+'_genre']] = 1

In [65]:
reviewsdf.shape

(5519, 90)

In [51]:
reviewsdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5519 entries, 0 to 5527
Data columns (total 90 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   title                       5519 non-null   object        
 1   num_reviews                 5519 non-null   int64         
 2   release_date                5519 non-null   datetime64[ns]
 3   orig_price                  5519 non-null   float64       
 4   been_sale                   5519 non-null   int64         
 5   review_rating               5519 non-null   object        
 6   rating_val                  5519 non-null   float64       
 7   num_languages               2101 non-null   float64       
 8   tags                        5519 non-null   object        
 9   num_tags                    5519 non-null   float64       
 10  genres                      5519 non-null   object        
 11  num_genres                  5519 non-null   float64     

# final df
was able to successfully create dummys for all categoricl columns by top ranks. saving this df as final df, saving a second df as modlingdf as the final df without wnanted columns.

In [66]:
reviewsdf.to_csv(r'/Users/michaelharnett/desktop/metis\finaldf.csv')

In [72]:
reviewsdf.Action_tag.sum()

0

In [53]:
reviewsdf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5519 entries, 0 to 5527
Data columns (total 90 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   title                       5519 non-null   object        
 1   num_reviews                 5519 non-null   int64         
 2   release_date                5519 non-null   datetime64[ns]
 3   orig_price                  5519 non-null   float64       
 4   been_sale                   5519 non-null   int64         
 5   review_rating               5519 non-null   object        
 6   rating_val                  5519 non-null   float64       
 7   num_languages               2101 non-null   float64       
 8   tags                        5519 non-null   object        
 9   num_tags                    5519 non-null   float64       
 10  genres                      5519 non-null   object        
 11  num_genres                  5519 non-null   float64     

In [67]:
modeldf = reviewsdf.drop(columns=['title','release_date', 'review_rating', 
                                  'tags', 'genres', 'publisher',
                                   'num_languages', 'top_pubs'])

In [68]:
modeldf.head(3)

,num_reviews,orig_price,been_sale,rating_val,num_tags,num_genres,year_released,languages_filled,BANDAI NAMCO Entertainment,Bethesda Softworks,...,Open_genre,FirstPerson_genre,Survival_genre,Funny_genre,Coop_genre,Fantasy_genre,Early_genre,Access_genre,Platformer_genre,Local_genre
0,111078,39.99,1,8.0,20.0,2.0,2020,13.0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,11813,39.99,0,9.0,20.0,3.0,2021,12.0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,216651,19.99,0,9.0,20.0,6.0,2021,12.0,0,0,...,0,0,0,0,0,0,1,1,0,0


In [69]:
modeldf.to_csv(r'/Users/michaelharnett/desktop/metis\modeldf.csv')

In [70]:
modeldf.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5519 entries, 0 to 5527
Data columns (total 82 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   num_reviews                 5519 non-null   int64  
 1   orig_price                  5519 non-null   float64
 2   been_sale                   5519 non-null   int64  
 3   rating_val                  5519 non-null   float64
 4   num_tags                    5519 non-null   float64
 5   num_genres                  5519 non-null   float64
 6   year_released               5519 non-null   int64  
 7   languages_filled            5519 non-null   float64
 8   BANDAI NAMCO Entertainment  5519 non-null   uint8  
 9   Bethesda Softworks          5519 non-null   uint8  
 10  Devolver Digital            5519 non-null   uint8  
 11  Disney                      5519 non-null   uint8  
 12  Electronic Arts             5519 non-null   uint8  
 13  Kagura Games                5519 

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5519 entries, 0 to 5527
Data columns (total 90 columns):
 #   Column                      Non-Null Count  Dtype         
---  ------                      --------------  -----         
 0   title                       5519 non-null   object        
 1   num_reviews                 5519 non-null   int64         
 2   release_date                5519 non-null   datetime64[ns]
 3   orig_price                  5519 non-null   float64       
 4   been_sale                   5519 non-null   int64         
 5   review_rating               5519 non-null   object        
 6   rating_val                  5519 non-null   float64       
 7   num_languages               2101 non-null   float64       
 8   tags                        5519 non-null   object        
 9   num_tags                    5519 non-null   float64       
 10  genres                      5519 non-null   object        
 11  num_genres                  5519 non-null   float64     